In [8]:
import pandas as pd
from Bio.KEGG import REST
from Bio.KEGG import Compound

numSheets = 1 
#have the user select the file they would like to have read in.
filename = 'ExampleEnzymeLookUp.xlsx'

#heatmapEnzyme Outputs
outFile = 'HeatmapEnzyme.xlsx'
writer = pd.ExcelWriter(outFile, engine='xlsxwriter')

for i in range(numSheets):
    #read in each sheet
    dataCur = pd.read_excel(filename,sheet_name=i)
    dFDict = {}
    #get the compounds and determine how many pathways hits there are.
    for j in range(len(dataCur['Cpd.Hits'])):
        #for the current compound hits find the length
        CpdList = dataCur['Cpd.Hits'][j]

        #split the current list by ;
        CpdList = CpdList.split(';')
        
        for k in range(len(CpdList)):
            #get the enzyme numbers from KEGG
            try:
                request = REST.kegg_get(CpdList[k])

            except:
                print("Cannot find compound, this should not happen")

            txtFCur = CpdList[k] + '.txt'
            open(txtFCur,'w').write(request.read())
            
            records = Compound.parse(open(txtFCur))

            #get the record of the compound currently being looked up.
            try:
                record = list(records)[0]
            except:
                print(': Almost for sure a glycan was found.')

            if k == 0 and j == 0:
                dict = {'Cluster #':[i+1],
                        'Pathway':dataCur['Pathway'][0],
                        'Pathway Total':dataCur['Pathway total'][0],
                        'Hits.total':dataCur['Hits.total'][0],
                        'Hits.sig':dataCur['Hits.sig'][0],
                        'Gamma':dataCur['Gamma'][0],
                        'Cpd.Hits':CpdList[0],
                        'Enzyme #s':['hi']
                        }
                #create a spreadsheet for the current hits
                dFDict[i] = pd.DataFrame(dict)
                dFDict[i]['Enzyme #s'][0] = record.enzyme
            
            elif k == 0 and j !=0:
                #create a spreadsheet for the current hits
                dFDict[i].loc[len(dFDict[i].index)] = [i+1,dataCur['Pathway'][j],dataCur['Pathway total'][j],dataCur['Hits.total'][j],dataCur['Hits.sig'][j],dataCur['Gamma'][j],CpdList[0], record.enzyme]

            else:
                dFDict[i].loc[len(dFDict[i].index)] = [None,None,None,None,None,None,CpdList[k],record.enzyme]

    dFDict[i].to_excel(writer,sheet_name=str(i+1))
try:
    writer.close()
except:
    print('No worky')

/var/folders/1d/ksvrky2d2gx960mpf77bmbxw0000gn/T/ipykernel_25580/2798581080.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dFDict[i]['Enzyme #s'][0] = record.enzyme


In [7]:
import pandas as pd

# Create a DataFrame
df = pd.DataFrame({'Data': [10, 20, 30, 40]})

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('Heatmap.xlsx', engine='xlsxwriter')

# Write the DataFrame to the Excel file
df.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and save the Excel file
writer.close()
